In [1]:
import chess
import chess.polyglot
import math
import random

In [2]:
# Define Styles
from IPython.core.display import HTML
with open('style.css','r') as file:
    css = file.read()
HTML(css)

# Evaluation of the given state
https://www.chessprogramming.org/Simplified_Evaluation_Function

## Hash function (zobrist hash)

In [3]:
# Maximum number of moves is 5898 (due to the 50 Moves Rule - that makes this number big enough)
WIN_VALUE = 2147483647
CACHED_VALUES = None
CACHED_VALUES_ACCESS_COUNT = 0
OVERALL_ACCESS_COUNT = 0
TABLE = [[random.randint(0, 2**64 - 1) for piece_value in range(1,13)] for square in range(1,65)]

def piece_key(piece):
    return (piece.piece_type + (6 if piece.color else 0)) - 1

def zobrist_hash(board):
    return chess.polyglot.zobrist_hash(board)

## Evaluation function

### Piece-Square Tables
Source: https://www.chessprogramming.org/Simplified_Evaluation_Function#Piece-Square_Tables

In [4]:
black_pawn_values = [0,  0,  0,  0,  0,  0,  0,  0,
                     50, 50, 50, 50, 50, 50, 50, 50,
                     10, 10, 20, 30, 30, 20, 10, 10,
                     5,  5, 10, 25, 25, 10,  5,  5,
                     0,  0,  0, 20, 20,  0,  0,  0,
                     5, -5, -10,  0,  0, -10, -5,  5,
                     5, 10, 10, -20, -20, 10, 10,  5,
                     0,  0,  0,  0,  0,  0,  0,  0]

black_knight_values = [-50, -40, -30, -30, -30, -30, -40, -50,
                       -40, -20,  0,  0,  0,  0, -20, -40,
                       -30,  0, 10, 15, 15, 10,  0, -30,
                       -30,  5, 15, 20, 20, 15,  5, -30,
                       -30,  0, 15, 20, 20, 15,  0, -30,
                       -30,  5, 10, 15, 15, 10,  5, -30,
                       -40, -20,  0,  5,  5,  0, -20, -40,
                       -50, -40, -30, -30, -30, -30, -40, -50]

black_bishop_values = [-20, -10, -10, -10, -10, -10, -10, -20,
                       -10,  0,  0,  0,  0,  0,  0, -10,
                       -10,  0,  5, 10, 10,  5,  0, -10,
                       -10,  5,  5, 10, 10,  5,  5, -10,
                       -10,  0, 10, 10, 10, 10,  0, -10,
                       -10, 10, 10, 10, 10, 10, 10, -10,
                       -10,  5,  0,  0,  0,  0,  5, -10,
                       -20, -10, -10, -10, -10, -10, -10, -20]

black_rook_values = [0,  0,  0,  0,  0,  0,  0,  0,
                     5, 10, 10, 10, 10, 10, 10,  5,
                     -5,  0,  0,  0,  0,  0,  0, -5,
                     -5,  0,  0,  0,  0,  0,  0, -5,
                     -5,  0,  0,  0,  0,  0,  0, -5,
                     -5,  0,  0,  0,  0,  0,  0, -5,
                     -5,  0,  0,  0,  0,  0,  0, -5,
                     0,  0,  0,  5,  5,  0,  0,  0]

black_queen_values = [-20, -10, -10, -5, -5, -10, -10, -20,
                      -10,  0,  0,  0,  0,  0,  0, -10,
                      -10,  0,  5,  5,  5,  5,  0, -10,
                      -5,  0,  5,  5,  5,  5,  0, -5,
                      0,  0,  5,  5,  5,  5,  0, -5,
                      -10,  5,  5,  5,  5,  5,  0, -10,
                      -10,  0,  5,  0,  0,  0,  0, -10,
                      -20, -10, -10, -5, -5, -10, -10, -20]

black_king_values_middlegame = [-30, -40, -40, -50, -50, -40, -40, -30,
                                -30, -40, -40, -50, -50, -40, -40, -30,
                                -30, -40, -40, -50, -50, -40, -40, -30,
                                -30, -40, -40, -50, -50, -40, -40, -30,
                                -20, -30, -30, -40, -40, -30, -30, -20,
                                -10, -20, -20, -20, -20, -20, -20, -10,
                                20, 20,  0,  0,  0,  0, 20, 20,
                                20, 30, 10,  0,  0, 10, 30, 20]

black_king_values_endgame = [-50, -40, -30, -20, -20, -30, -40, -50,
                             -30, -20, -10,  0,  0, -10, -20, -30,
                             -30, -10, 20, 30, 30, 20, -10, -30,
                             -30, -10, 30, 40, 40, 30, -10, -30,
                             -30, -10, 30, 40, 40, 30, -10, -30,
                             -30, -10, 20, 30, 30, 20, -10, -30,
                             -30, -30,  0,  0,  0,  0, -30, -30,
                             -50, -30, -30, -30, -30, -30, -30, -50]


# Evaluation functions

## Auxiliary functions

`is_endgame` takes the current `board` and returns with help of `color_endgame` wether both players match the criteria for the endgame according to the simple evaluation function. (See https://www.chessprogramming.org/Simplified_Evaluation_Function#King).

The functions will be used for the evaluation of the king's position using the piece-square table.

In [5]:
def color_endgame(board, color):
    return len(board.pieces(chess.QUEEN, color)) == 0 or (len(board.pieces(chess.BISHOP, color)) + len(board.pieces(chess.KNIGHT, color)) + len(board.pieces(chess.ROOK, color))) < 2

def is_endgame(board):
    white_endgame = color_endgame(board, chess.WHITE)
    black_endgame = color_endgame(board, chess.BLACK)
    return white_endgame and black_endgame

## Raw evaluation function

The function takes the parameters `board` (the current board) and `endgame` (if the players are in the endgame) and evaluates the current state of the board according to the simplified evaluation function.

In [6]:
def static_eval(board, endgame):
    # Check if game is over
    if (board.result() == "1-0" and board.turn) or (board.result() == "0-1" and not board.turn):
        result = WIN_VALUE - board.fullmove_number
        return result
    elif (board.result() == "0-1" and board.turn) or (board.result() == "1-0" and not board.turn):
        result = -WIN_VALUE + board.fullmove_number
        return result
    elif board.result() == "1/2-1/2" or board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
        result = 0
        return result
    
    value = 0
    
    # Piece values, numbers represent centipawns; Pawn, Knight, Bishop, Rook, Queen
    # Source: https://www.chessprogramming.org/Simplified_Evaluation_Function#Piece_Values
    piece_values = [100, 320, 330, 500, 900]
    
    for piece_type, piece_value in enumerate(piece_values):
        piece_type += 1
        
        player_pieces = board.pieces(piece_type, board.turn)
        enemy_pieces = board.pieces(piece_type, not board.turn)
        
        value += len(player_pieces)*piece_value
        value -= len(enemy_pieces)*piece_value
        
        if piece_type == chess.PAWN:
            value += piece_square_tables(black_pawn_values, player_pieces, enemy_pieces, board)
            
        if piece_type == chess.KNIGHT:
            value += piece_square_tables(black_knight_values, player_pieces, enemy_pieces, board)
            
        if piece_type == chess.BISHOP:
            value += piece_square_tables(black_bishop_values, player_pieces, enemy_pieces, board)
            
        if piece_type == chess.ROOK:
            value += piece_square_tables(black_rook_values, player_pieces, enemy_pieces, board)
            
        if piece_type == chess.QUEEN:
            value += piece_square_tables(black_queen_values, player_pieces, enemy_pieces, board)
            
        if piece_type == chess.KING:
            if endgame:
                value += piece_square_tables(black_king_values_endgame, player_pieces, enemy_pieces, board)
            else:
                value += piece_square_tables(black_king_values_middlegame, player_pieces, enemy_pieces, board)
    return value

### Function for the Piece-Square Tables

This function calculates the value using `black_piece_values` (according to the piece-square tables), `player_pieces` and `enemy_pieces` (the list of the player's respectively the opponent's pieces) as well as the current `board` and returns the calculated value which will be added to the already calculated one. 

In [7]:
def piece_square_tables(black_piece_values, player_pieces, enemy_pieces, board):
    value = 0
    white_piece_values = black_piece_values[::-1]
    
    for piece in player_pieces:
        if board.turn:
            value += white_piece_values[piece]
        else:
            value += black_piece_values[piece]
            
    for piece in enemy_pieces:
        if board.turn:
            value -= black_piece_values[piece]
        else:
            value -= white_piece_values[piece]
            
    return value

## Main minimax function

The main function implementing the minimax algorithm takes the following parameters:

- `board`, the current board
- `depth`, the depth to which the board will be analyzed
- `alpha` and `beta`, needed for Alpha-Beta-Pruning

The function returns the biggest calculated value and it sets the global variable `BEST_MOVE` which is the move to play.  
The calculated value is also being added to the cache entries or, if it already exists, directly returns the value from the cache.

In [8]:
# TODO DOCUMENTATION!!!1!elf
# key: (zobrist_hash(board), depth)
# value: (value, uci_move, alpha, beta)
cache = {}

In [13]:
ANALYZING_DEPTH = None

def cached_minimax(board, depth, alpha, beta):
    global BEST_MOVE
    
    cache_key = (zobrist_hash(board), depth)
    
    if cache_key in cache.keys():
        c_value, c_uci_move, c_alpha, c_beta = cache[cache_key]
        
        if c_alpha <= alpha and beta <= c_beta:
            BEST_MOVE = Move.from_uci(c_uci_move)
            return c_value
        
        else:            
            min_alpha = min(alpha, c_alpha)
            max_beta = max(beta, c_beta)
            value = minimax(board, depth, min_alpha, max_beta) # BEST_MOVE is set here
            cache[cache_key] = (value, BEST_MOVE.uci, min_alpha, max_beta)
            return value
    else:
        value = minimax(board, depth, alpha, beta) # BEST_MOVE is set here
        cache[cache_key] = (value, BEST_MOVE.uci, alpha, beta)
        return value

In [14]:
def minimax(board, depth, alpha, beta):
    global BEST_MOVE
    
    if (depth == 0 or not board.legal_moves):
        value = static_eval(board, is_endgame(board))
        return value
    
    max_value = alpha
    
    for move in board.legal_moves:
        board.push(move)
        value = -minimax(board, depth - 1, -beta, -max_value)
        board.pop()
        if (value > max_value):
            max_value = value
            if (depth == ANALYZING_DEPTH):
                BEST_MOVE = move;
            if (max_value >= beta):
                break
    
    return max_value

### Input function for minimax
This function takes the current `board` and the desired `depth` and sets global variables for caching (`CACHED_VALUES` and `CACHED_VALUES_EVAL`) and debugging (`STATIC_EVAL_EXECUTIONS`, `MINIMAX_EXECUTIONS`, and `OVERALL_MINIMAX_EXECUTIONS`) purposes. It checks, whether the move is in the opening library and, if so, returns the documented move. If the move is not in the opening library, it calls `minimax` with the correct parameters. 

In [11]:
BEST_MOVE = None

def minimax_input(board, depth):
    global ANALYZING_DEPTH
    ANALYZING_DEPTH = depth
    
    value, polyglot_move = get_polyglot_move(board)
    if polyglot_move:
        return value, polyglot_move
    
    return cached_minimax(board, depth, -math.inf, math.inf), BEST_MOVE

### Using the opening library
This auxiliary function takes the current `board` and checks if the played move is in an external opening library. It then chooses a random move, proposed by the library and returns the value 0 and the `move`.

In [12]:
POLYGLOT_PATH = "data/polyglot/performance.bin"
def get_polyglot_move(board):
    
    with chess.polyglot.open_reader(POLYGLOT_PATH) as reader:
        
        for entry in set(reader.find_all(board)):
                return 0, entry.move
            
        return 0, None